In [ ]:
"""
=============================================================================
SISTEM PEMILAH SAMPAH CERDAS - TRAINING YOLOV8M
Google Colab Training Script
=============================================================================

CARA PAKAI:
1. Upload file ini ke Google Colab
2. Ganti YOUR_ROBOFLOW_API_KEY dengan API key Anda
3. Run semua cell
4. Model akan tersimpan otomatis di Google Drive

Author: @krompium
Untuk: Tugas UAS
=============================================================================
"""

# ============================================================================
# CELL 1: Setup Environment & Check GPU
# ============================================================================
print("="*70)
print("🗑️  SISTEM PEMILAH SAMPAH CERDAS - YOLOV8M TRAINING")
print("="*70)
print("\n📋 Step 1: Checking GPU & Installing Dependencies...")

# Check GPU
import subprocess
result = subprocess.run(['nvidia-smi'], capture_output=True, text=True)
print(result.stdout)

# Install packages
print("\n📦 Installing required packages...")
!pip install ultralytics roboflow -q

# Import libraries
import os
import shutil
from pathlib import Path
import yaml
from IPython.display import Image, display, clear_output
from ultralytics import YOLO
import torch
import matplotlib.pyplot as plt
import random
from PIL import Image as PILImage
from tqdm.auto import tqdm

# Check setup
print(f"\n✅ Setup Complete!")
print(f"   PyTorch version: {torch.__version__}")
print(f"   CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"   GPU: {torch.cuda.get_device_name(0)}")
    print(f"   VRAM: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")
else:
    print("   ⚠️  WARNING: GPU tidak terdeteksi!  Training akan sangat lambat.")

# ============================================================================
# CELL 2: Mount Google Drive
# ============================================================================
print("\n" + "="*70)
print("📁 Step 2: Mounting Google Drive...")
print("="*70)

from google.colab import drive
drive.mount('/content/drive')

# Buat folder project di Google Drive
PROJECT_DIR = Path('/content/drive/MyDrive/sistem-pemilah-sampah')
PROJECT_DIR.mkdir(exist_ok=True)

print(f"\n✅ Google Drive mounted!")
print(f"   Project directory: {PROJECT_DIR}")

# ============================================================================
# CELL 3: Download Dataset dari Roboflow
# ============================================================================
print("\n" + "="*70)
print("📥 Step 3: Downloading Dataset dari Roboflow...")
print("="*70)

from roboflow import Roboflow

# ⚠️ GANTI DENGAN API KEY ANDA!
# Dapatkan di: https://app.roboflow.com/settings/api
ROBOFLOW_API_KEY = "YOUR_ROBOFLOW_API_KEY_HERE"  # <-- GANTI INI!

if ROBOFLOW_API_KEY == "YOUR_ROBOFLOW_API_KEY_HERE":
    print("\n❌ ERROR: Roboflow API Key belum diganti!")
    print("   Silakan daftar di https://roboflow.com")
    print("   Dapatkan API key di Settings → API")
    print("   Lalu ganti di code di atas")
else:
    try:
        # Initialize Roboflow
        rf = Roboflow(api_key=ROBOFLOW_API_KEY)

        # Download dataset
        print("\n📥 Downloading dataset (ini mungkin memakan waktu beberapa menit)...")
        project = rf.workspace("siscer-project").project("sampah-organik-dan-anorganik")
        dataset = project.version(3).download("yolov8")

        DATASET_PATH = dataset.location
        print(f"\n✅ Dataset berhasil didownload!")
        print(f"   Location: {DATASET_PATH}")

    except Exception as e:
        print(f"\n❌ Error downloading dataset: {e}")
        print("   Pastikan API key sudah benar")
        print("   Pastikan koneksi internet stabil")

# ============================================================================
# CELL 4: Explorasi Dataset
# ============================================================================
print("\n" + "="*70)
print("🔍 Step 4: Exploring Dataset...")
print("="*70)

# Check struktur dataset
print("\n📂 Dataset structure:")
for root, dirs, files in os.walk(DATASET_PATH):
    level = root.replace(DATASET_PATH, '').count(os.sep)
    indent = '  ' * level
    print(f"{indent}{os.path.basename(root)}/")
    subindent = '  ' * (level + 1)
    for file in files[: 3]:
        print(f"{subindent}{file}")
    if len(files) > 3:
        print(f"{subindent}... dan {len(files)-3} file lainnya")
    if level > 2:  # Batasi kedalaman
        break

# Hitung jumlah gambar
train_images = list(Path(DATASET_PATH).glob('train/images/*. jpg')) + \
               list(Path(DATASET_PATH).glob('train/images/*.png'))
val_images = list(Path(DATASET_PATH).glob('valid/images/*.jpg')) + \
             list(Path(DATASET_PATH).glob('valid/images/*.png'))

print(f"\n📊 Dataset Statistics:")
print(f"   Training images  : {len(train_images)}")
print(f"   Validation images: {len(val_images)}")
print(f"   Total images     : {len(train_images) + len(val_images)}")

# Tampilkan sample images
print("\n🖼️  Sample images dari dataset:")
fig, axes = plt.subplots(2, 3, figsize=(15, 10))
axes = axes.ravel()

sample_images = random.sample(train_images, min(6, len(train_images)))
for idx, img_path in enumerate(sample_images):
    img = PILImage.open(img_path)
    axes[idx].imshow(img)
    axes[idx].set_title(img_path.name, fontsize=10)
    axes[idx].axis('off')

plt.tight_layout()
plt.savefig('dataset_samples.png', dpi=150, bbox_inches='tight')
print("   ✅ Sample images tersimpan sebagai 'dataset_samples.png'")
plt.show()

# ============================================================================
# CELL 5: Update data. yaml untuk 3 Kelas
# ============================================================================
print("\n" + "="*70)
print("⚙️  Step 5: Configuring Dataset...")
print("="*70)

yaml_path = Path(DATASET_PATH) / 'data.yaml'

# Baca data.yaml yang ada
with open(yaml_path, 'r') as f:
    data_config = yaml.safe_load(f)

print("\n📄 Original data.yaml:")
print(yaml. dump(data_config, default_flow_style=False))

# Jika ingin update ke 3 kelas (organik, anorganik, b3)
# Uncomment jika dataset sudah ada kelas B3:
# data_config['nc'] = 3
# data_config['names'] = ['organik', 'anorganik', 'b3']
# with open(yaml_path, 'w') as f:
#     yaml.dump(data_config, f, default_flow_style=False)
# print("\n✅ data.yaml updated untuk 3 kelas")

print("\n✅ Dataset configuration ready!")

# ============================================================================
# CELL 6: Training YOLOv8m
# ============================================================================
print("\n" + "="*70)
print("🚀 Step 6: Training YOLOv8m Model...")
print("="*70)

# Training configuration
CONFIG = {
    'model': 'yolov8m. pt',       # YOLOv8 Medium
    'epochs': 200,                # Epochs (bisa dikurangi jika mau cepat)
    'batch': 32,                  # Batch size untuk Colab Pro
    'imgsz': 640,                 # Image size
    'patience': 50,               # Early stopping
    'device': 0,                  # GPU
    'project': 'sampah-3kelas',
    'name': 'yolov8m_training',
}

print("\n⚙️  Training Configuration:")
for key, value in CONFIG.items():
    print(f"   {key: 12s}: {value}")

# Load model
print("\n📥 Loading YOLOv8m pretrained model...")
model = YOLO(CONFIG['model'])

print("\n🏋️  Starting training (ini akan memakan waktu 1-2 jam)...")
print("   Anda bisa minimize tab ini, training akan jalan otomatis")
print("   " + "="*66)

# Train
results = model.train(
    data=str(yaml_path),
    epochs=CONFIG['epochs'],
    imgsz=CONFIG['imgsz'],
    batch=CONFIG['batch'],
    device=CONFIG['device'],
    patience=CONFIG['patience'],
    project=CONFIG['project'],
    name=CONFIG['name'],

    # Optimization
    optimizer='AdamW',
    lr0=0.001,
    lrf=0.01,
    momentum=0.937,
    weight_decay=0.0005,
    warmup_epochs=5,
    warmup_momentum=0.8,
    warmup_bias_lr=0.1,

    # Data Augmentation
    hsv_h=0.015,
    hsv_s=0.7,
    hsv_v=0.4,
    degrees=10. 0,
    translate=0.1,
    scale=0.5,
    shear=2.0,
    perspective=0.0,
    flipud=0.0,
    fliplr=0.5,
    mosaic=1.0,
    mixup=0.15,
    copy_paste=0.1,

    # Saving
    save=True,
    save_period=20,
    plots=True,
    verbose=True,
)

print("\n" + "="*70)
print("✅ TRAINING SELESAI!")
print("="*70)

# ============================================================================
# CELL 7: Evaluasi Model
# ============================================================================
print("\n" + "="*70)
print("📊 Step 7: Evaluating Model...")
print("="*70)

# Evaluate
metrics = model.val()

# Display metrics
print("\n📈 EVALUATION RESULTS:")
print("="*70)
print(f"mAP50        : {metrics.box.map50:. 4f}")
print(f"mAP50-95     : {metrics.box.map:. 4f}")
print(f"Precision    : {metrics.box. mp:.4f}")
print(f"Recall       : {metrics.box.mr:.4f}")
print("="*70)

# Per-class metrics
if hasattr(metrics.box, 'ap50') and len(metrics.box.ap50) > 0:
    print("\n📊 Per-Class Performance:")
    class_names = data_config['names']
    for i, name in enumerate(class_names):
        if i < len(metrics.box.ap50):
            print(f"   {name: 12s}: mAP50 = {metrics. box.ap50[i]:. 4f}")

# ============================================================================
# CELL 8: Visualisasi Hasil Training
# ============================================================================
print("\n" + "="*70)
print("📊 Step 8: Visualizing Results...")
print("="*70)

run_dir = Path(CONFIG['project']) / CONFIG['name']

# Training plots
results_img = run_dir / 'results.png'
if results_img.exists():
    print("\n📈 Training Metrics:")
    display(Image(filename=str(results_img)))

# Confusion matrix
confusion_img = run_dir / 'confusion_matrix.png'
if confusion_img.exists():
    print("\n🎯 Confusion Matrix:")
    display(Image(filename=str(confusion_img)))

# Validation predictions
val_batch = run_dir / 'val_batch0_pred.jpg'
if val_batch.exists():
    print("\n🔍 Sample Predictions:")
    display(Image(filename=str(val_batch)))

# ============================================================================
# CELL 9: Test Inference
# ============================================================================
print("\n" + "="*70)
print("🧪 Step 9: Testing Inference...")
print("="*70)

# Load best model
best_model_path = run_dir / 'weights' / 'best.pt'
best_model = YOLO(str(best_model_path))

print(f"\n✅ Best model loaded dari: {best_model_path}")

# Test pada random validation images
print("\n🔍 Testing pada sample validation images...")

test_images = random.sample(val_images, min(4, len(val_images)))

fig, axes = plt.subplots(2, 2, figsize=(15, 15))
axes = axes.ravel()

for idx, img_path in enumerate(test_images):
    # Predict
    results = best_model(str(img_path), verbose=False)

    # Plot
    annotated = results[0].plot()
    axes[idx].imshow(annotated[.. ., : :-1])  # BGR to RGB
    axes[idx]. set_title(f"Prediction: {img_path.name}", fontsize=10)
    axes[idx].axis('off')

    # Print detections
    if len(results[0].boxes) > 0:
        for box in results[0].boxes:
            cls = int(box.cls[0])
            conf = float(box.conf[0])
            class_name = data_config['names'][cls] if cls < len(data_config['names']) else f"Class{cls}"
            print(f"   {img_path.name}:  {class_name} ({conf:.2%})")

plt.tight_layout()
plt.savefig('test_predictions.png', dpi=150, bbox_inches='tight')
print("\n   ✅ Predictions tersimpan sebagai 'test_predictions.png'")
plt.show()

# ============================================================================
# CELL 10: Export Model
# ============================================================================
print("\n" + "="*70)
print("📦 Step 10: Exporting Model...")
print("="*70)

print("\n📤 Exporting model ke berbagai format...")

# Export to ONNX
print("\n1️⃣  Exporting to ONNX...")
try:
    best_model.export(format='onnx', imgsz=640)
    print("   ✅ ONNX export complete")
except Exception as e:
    print(f"   ⚠️  ONNX export failed: {e}")

# Export to TFLite
print("\n2️⃣  Exporting to TensorFlow Lite...")
try:
    best_model.export(format='tflite', imgsz=640)
    print("   ✅ TFLite export complete")
except Exception as e:
    print(f"   ⚠️  TFLite export failed: {e}")

# Export to TorchScript
print("\n3️⃣  Exporting to TorchScript...")
try:
    best_model.export(format='torchscript', imgsz=640)
    print("   ✅ TorchScript export complete")
except Exception as e:
    print(f"   ⚠️  TorchScript export failed: {e}")

print("\n✅ Model export selesai!")

# ============================================================================
# CELL 11: Copy ke Google Drive
# ============================================================================
print("\n" + "="*70)
print("💾 Step 11: Saving to Google Drive...")
print("="*70)

# Copy semua hasil ke Google Drive
drive_save_path = PROJECT_DIR / CONFIG['name']

print(f"\n📁 Copying results to Google Drive...")
print(f"   From: {run_dir}")
print(f"   To  : {drive_save_path}")

if drive_save_path.exists():
    print(f"   ⚠️  Folder sudah ada, menghapus yang lama...")
    shutil.rmtree(drive_save_path)

shutil.copytree(run_dir, drive_save_path)

print(f"\n✅ Results berhasil disimpan ke Google Drive!")
print(f"   Location: {drive_save_path}")

print(f"\n📁 File yang tersimpan:")
print(f"   ✅ best. pt - Model terbaik (PyTorch)")
print(f"   ✅ last.pt - Model terakhir (checkpoint)")
print(f"   ✅ best. onnx - ONNX format")
print(f"   ✅ best.tflite - TensorFlow Lite")
print(f"   ✅ results.png - Training plots")
print(f"   ✅ confusion_matrix.png - Confusion matrix")
print(f"   ✅ Dan file lainnya...")

# ============================================================================
# CELL 12: Download Model
# ============================================================================
print("\n" + "="*70)
print("⬇️  Step 12: Download Model (Optional)")
print("="*70)

from google.colab import files

print("\n📥 Apakah Anda ingin download model sekarang?")
print("   Model sudah tersimpan di Google Drive, tapi Anda juga")
print("   bisa download langsung untuk backup.")
print()

download_now = input("Download sekarang? (y/n): ").lower().strip()

if download_now == 'y':
    # Download best.pt
    best_pt = run_dir / 'weights' / 'best.pt'
    if best_pt.exists():
        print(f"\n⬇️  Downloading best.pt...")
        files.download(str(best_pt))
        print("   ✅ Download complete!")

    # Download ONNX
    best_onnx = run_dir / 'weights' / 'best.onnx'
    if best_onnx.exists():
        print(f"\n⬇️  Downloading best.onnx...")
        files.download(str(best_onnx))
        print("   ✅ Download complete!")
else:
    print("\n✅ OK, model tetap tersimpan di Google Drive")
    print(f"   Anda bisa download nanti dari: {drive_save_path}/weights/")

# ============================================================================
# CELL 13: Summary
# ============================================================================
print("\n" + "="*70)
print("🎉 TRAINING SELESAI - SUMMARY")
print("="*70)

print(f"\n📊 Project     : {CONFIG['project']}")
print(f"📝 Name        : {CONFIG['name']}")
print(f"🤖 Model       : {CONFIG['model']}")
print(f"📈 Epochs      : {CONFIG['epochs']}")
print(f"📊 Dataset     : {len(train_images)} train, {len(val_images)} val")

print(f"\n📈 Final Metrics:")
print(f"   mAP50      : {metrics.box.map50:.4f}")
print(f"   mAP50-95   : {metrics.box.map:.4f}")
print(f"   Precision  : {metrics.box.mp:.4f}")
print(f"   Recall     : {metrics.box.mr:.4f}")

print(f"\n💾 Model Location:")
print(f"   Google Drive: {drive_save_path}")
print(f"   Colab       : {run_dir}")

print(f"\n📁 File Penting:")
print(f"   ✅ {drive_save_path}/weights/best.pt")
print(f"   ✅ {drive_save_path}/weights/best.onnx")
print(f"   ✅ {drive_save_path}/weights/best.tflite")

print("\n🚀 NEXT STEPS:")
print("=" *70)
print("1. Download best.pt dari Google Drive")
print("2. Copy ke folder models/ di repository")
print("3. Run inference script di laptop")
print("4. Upload code ke ESP32 & ESP32-CAM")
print("5. Setup Blynk")
print("6. Test & demo untuk UAS!")
print("="*70)

print("\n✅ Selamat!  Training berhasil!")
print("   Good luck untuk UAS besok!  🎓🔥")
print("="*70)